In [1]:
import boto3
import os
from config import ACCESS_KEY, SECRET_KEY, TOKEN
import datetime
import pytz

import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
token = TOKEN
headers={'Authorization':token,'Accept':'application/json'}

#'b1gb310irjlk6b99e14g' - аналитика
#'b1gc7vi2ckqausoc5dr7' - спутник

FOLDER_ID = 'b1gc7vi2ckqausoc5dr7' # id каталога из которого береться запрос
ACCESS_KEY = ACCESS_KEY #aws_access_key_id для S3
SECRET_KEY = SECRET_KEY #aws_secret_access_key в s3



In [3]:
bucket_name = 'dwh-asgard' # имя бакета

In [4]:
#создаем подключение к yndex storage через .session.Session()

def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )

s3 = get_s3_instance()

### Выгрузка ключей файлов из определенной папки

Если нужно проверить целостность данных, собираемых по дням. 

In [ ]:
#StartAfter позволяет сделать запрос с определенного ключа

s3.list_objects_v2(Bucket=bucket_name,StartAfter = 'citizen_payments_st_mobile/year=2023/month=07/15.csv')

In [ ]:
#код позволяет посмотеть список папок

s3.list_objects_v2(Bucket=bucket_name, Delimiter='/')['CommonPrefixes'] 

In [6]:

# Если нужны все файлы, то не прописываем Prefix то Prefix=False
def list_of_daily_objects_from_s3(bucket_name, Prefix):
    s3 = get_s3_instance()
    # Создаем пагинатор. Он нужен на тот случай, если файлов больше 1000
    paginator = s3.get_paginator('list_objects_v2')

    # Присваеваем пагинатор с параметрами. 
    if folder is False:
        page_iterator = paginator.paginate(Bucket=bucket_name)
    else:    
        page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=Prefix)

    df_obj = pd.DataFrame({
        'folder': [],
        'year': [],
        'month': [],
        'day': [],
        'key':[],
        'LastModified':[],
        }) # создаем пустой датасет для анализа

    obj_number = 0 
    for page in page_iterator:
        for obj in page['Contents']: # list_objects - список объектов в бакете, Prefix - поиск во ключевому слову
            if len(obj['Key'].split(sep='/')) == 4 \
            and obj['Key'].split(sep='/')[3]!='':   # условия фильтрации
                df_obj.loc[obj_number,['folder']] = obj['Key'].split(sep='/')[0]
                df_obj.loc[obj_number,['year']] = obj['Key'].split(sep='/')[1].replace('year=','')
                df_obj.loc[obj_number,['month']] = obj['Key'].split(sep='/')[2].replace('month=','')
                df_obj.loc[obj_number,['day']] = obj['Key'].split(sep='/')[3].replace('.csv','')
                df_obj.loc[obj_number,['key']] = obj['Key']
                df_obj.loc[obj_number,['LastModified']] = obj['LastModified'].strftime('%Y-%m-%d %H:%M:%S')
            obj_number += 1 
                # добавляем в датасет найденные значения
                
    df_obj['date'] = df_obj['year']+'-'+df_obj['month']+'-'+df_obj['day']
    df_obj['date'] = pd.to_datetime(df_obj['date'], dayfirst=True)

    df_obj = df_obj.reset_index().drop('index', axis=1)
    return df_obj

In [34]:
folder = 'entries' # имя папки в бакете
Prefix = f"{folder}/"
df_obj = list_of_daily_objects_from_s3(bucket_name,Prefix)

In [35]:
df_obj

,folder,year,month,day,key,LastModified,date
0,entries,2023,04,28,entries/year=2023/month=04/28.csv,2023-04-28 01:04:14,2023-04-28
1,entries,2023,04,29,entries/year=2023/month=04/29.csv,2023-04-29 01:03:44,2023-04-29
2,entries,2023,04,30,entries/year=2023/month=04/30.csv,2023-04-30 01:07:22,2023-04-30
3,entries,2023,05,01,entries/year=2023/month=05/01.csv,2023-05-01 01:16:20,2023-05-01
4,entries,2023,05,02,entries/year=2023/month=05/02.csv,2023-05-02 01:04:00,2023-05-02
...,...,...,...,...,...,...,...
344,entries,2024,04,06,entries/year=2024/month=04/06.csv,2024-04-10 14:31:31,2024-04-06
345,entries,2024,04,07,entries/year=2024/month=04/07.csv,2024-04-07 02:44:43,2024-04-07
346,entries,2024,04,08,entries/year=2024/month=04/08.csv,2024-04-08 02:53:10,2024-04-08
347,entries,2024,04,09,entries/year=2024/month=04/09.csv,2024-04-09 02:41:26,2024-04-09


## dataframe пропущенных дат

In [15]:
def list_of_missing_data(df_obj):
    s3 = get_s3_instance()
    # задаем стартовую и финишную дату на основе ключей
    start_date = df_obj.loc[0,['date']].values[0]
    end_date = df_obj.loc[df_obj.shape[0],['date']].values[0]

    # создаем датасет с всеми датами
    dates_pd = pd.DataFrame({
        'date_range': pd.date_range(start=start_date, end=end_date),
        })

    # соединяем ранее полученный список и полный список дат
    dates_merged = dates_pd.merge(
        df_obj,
        left_on='date_range',
        right_on='date',
        how='left'
    )

    # фильтруем полученный список по null строкам и выносим даты в отедльный датафрейм
    df_of_missing_dates = pd.DataFrame()
    df_of_missing_dates['date'] = dates_merged[dates_merged['key'].isnull()]['date_range'] 
    df_of_missing_dates = df_of_missing_dates.reset_index().drop('index', axis=1)

    list_of_missing_dates = []

    # Подставляем под пропущенные даты ключ файла, из которого будет браться информация
    for i in range(df_of_missing_dates.shape[0]):

        next_date_day = (df_of_missing_dates.loc[i,['date']] + datetime.timedelta(days=1)).dt.strftime('%d').values[0]
        next_date_month = (df_of_missing_dates.loc[i,['date']] + datetime.timedelta(days=1)).dt.strftime('%m').values[0]
        next_date_year = (df_of_missing_dates.loc[i,['date']] + datetime.timedelta(days=1)).dt.strftime('%Y').values[0] 
        s3_file_name = f'{folder}/year={next_date_year}/month={next_date_month}/{next_date_day}.csv'
        try: 
            if s3.head_object(Bucket=bucket_name,Key=s3_file_name)['ResponseMetadata']['HTTPStatusCode'] == 200:
                df_of_missing_dates.loc[i,['next file key']] = s3_file_name
        except:
            continue
            
    # заполняем оставшиеся пропуски следующим не пустым значением
    # эта таблица  поможет выполнить дальнейшие действия
    df_of_missing_dates = df_of_missing_dates.fillna(method='bfill')
    return df_of_missing_dates

In [36]:
df_of_missing_dates =  list_of_missing_data(df_obj)

In [37]:
df_of_missing_dates 

,date


### Скачивание файлов

In [38]:
# В качестве ключей для скачивания используем ключи всех файлов из папки, полученные ранее

def downloading_dayly_files_from_s3(path_on_pc,df_obj):
    s3 = get_s3_instance()
    for i in range(0,df_obj.shape[0]):
        s3_file_name = df_obj.loc[i,['key']].values[0]
        s3_file_name_folder = '/'.join(s3_file_name.split(sep='/')[0:3])
        
        local_download_folder = f'{path_on_pc}/{s3_file_name_folder}'
        local_download_path = f'{path_on_pc}/{s3_file_name}' 
        
    # если данный файл уже скачан, он не скачивается повторно
        if os.path.isfile(local_download_path) is False:
        # создается новая папка, если ее нет
            os.makedirs(local_download_folder, exist_ok=True)
            s3.download_file(Bucket=bucket_name,Key=s3_file_name,Filename=local_download_path)


In [39]:
# путь к основной папки на компе
path_on_pc = 'C:/Users/Boris/Desktop/s3' 
downloading_dayly_files_from_s3(path_on_pc,df_obj)      

#### быстро делаем резервную копию папки перед обработкой

In [43]:
import shutil

src_folder = f"{path_on_pc}/{folder}"
dst_folder = f"{path_on_pc}/{folder}-{datetime.datetime.now().date().strftime('%Y-%m-%d')}"

shutil.copytree(src_folder, dst_folder)

'C:/Users/Boris/Desktop/s3/entries_st_mobile-2024-04-09'

#### Проссматриваем структуру данных для понимания как дата соотносится с названием

In [60]:
file_df = pd.read_csv(f'{path_on_pc}/{df_obj.loc[0,["key"]].values[0]}')
file_df.head()

,report_date,address_uuid,locality,address,intercom_count,partner_uuid,subsc_restrict,sponsored,archive_3,archive_5,archive_7,archive_0,subscriptions_standard_month,flats_total,flats_using_app,flats_active,flats_app_banned
0,2023-04-28,175394ea-2ece-4b23-9d81-cb2fa8f4106e,Тюмень,"Тюмень, улица 50 лет Октября, д. 60, п. 4",1,adeffb47-1707-4976-a4a6-e6eacf19b050,freemonetization,0,0,0,0,0,0,18,2,0,0
1,2023-04-28,c50640dd-49a2-4300-bfbc-082e015124be,Нур-Султан (Астана),"Нур-Султан (Астана), улица Е 49, д. 3/3, п. 2",2,af8b5bf4-0108-4a0c-acc2-096f94959fb3,stricted,0,0,0,0,0,0,87,35,0,0
2,2023-04-28,812e6d6f-e573-46de-b7f5-0481c325be9b,Волгоград,"Волгоград, улица Маршала Чуйкова, д. 9, п. 3",1,e27b624d-4c0e-48cc-bbc3-a0060ea0094c,freemonetization,0,0,0,0,0,0,100,3,0,0
3,2023-04-28,543cbc32-f744-40cb-9dcd-48b37c3a96bc,Тюмень,"Тюмень, улица Севастопольская, д. 33, п. 2",1,adeffb47-1707-4976-a4a6-e6eacf19b050,freemonetization,0,0,0,0,0,0,20,4,0,0
4,2023-04-28,525b5d32-707d-4e27-b000-9cbc1547351e,Тюмень,"Тюмень, улица Севастопольская, д. 25, п. 1",1,adeffb47-1707-4976-a4a6-e6eacf19b050,freemonetization,0,0,0,0,0,0,20,10,2,0


### Добавление пропущенных значений 

In [70]:
def adding_missing_data_to_pc(folder, path_on_pc, df_of_missing_dates):
    # проходимся по датафрейму пропущенных значений
    for i in range(df_of_missing_dates.shape[0]):
        new_date = df_of_missing_dates.loc[i,["date"]].values[0].strftime('%Y-%m-%d')
        df_of_missing_dates.loc[i,["date"]].values[0].strftime('year=%Y/month=%m/%d.csv')
        temp_df = pd.read_csv(f'{path_on_pc}/{df_of_missing_dates.loc[i,["next file key"]].values[0]}')
        # Обновляем дату
        temp_df['report_date'] = new_date
        temp_df.to_csv(f"{path_on_pc}/{folder}/{df_of_missing_dates.loc[i,['date']].values[0].strftime('year=%Y/month=%m/%d.csv')}" ,sep=',',index=False)
        # Создаем внутри датасета отдельный список ключей для загрузки
        df_of_missing_dates.loc[i,['new_key']] = f"{folder}/{df_of_missing_dates.loc[i,['date']].values[0].strftime('year=%Y/month=%m/%d.csv')}"



In [71]:
adding_missing_data_to_pc(folder, path_on_pc, df_of_missing_dates)

In [72]:
df_of_missing_dates

,date,next file key,new_key
0,2023-12-04,entries/year=2023/month=12/05.csv,entries/year=2023/month=12/04.csv
1,2023-12-06,entries/year=2023/month=12/07.csv,entries/year=2023/month=12/06.csv
2,2023-12-12,entries/year=2023/month=12/13.csv,entries/year=2023/month=12/12.csv
3,2023-12-14,entries/year=2023/month=12/15.csv,entries/year=2023/month=12/14.csv
4,2023-12-21,entries/year=2023/month=12/22.csv,entries/year=2023/month=12/21.csv
5,2023-12-28,entries/year=2023/month=12/30.csv,entries/year=2023/month=12/28.csv
6,2023-12-29,entries/year=2023/month=12/30.csv,entries/year=2023/month=12/29.csv
7,2024-01-19,entries/year=2024/month=01/20.csv,entries/year=2024/month=01/19.csv
8,2024-01-22,entries/year=2024/month=01/23.csv,entries/year=2024/month=01/22.csv
9,2024-02-01,entries/year=2024/month=02/02.csv,entries/year=2024/month=02/01.csv


In [80]:
def upploading_missing_data_to_storage(bucket_name,path_on_pc):
    s3 = get_s3_instance()
    for i in range(0,df_of_missing_dates.shape[0]):
        s3_file_path = df_of_missing_dates.loc[i,['new_key']].values[0]
        pc_file_path = f'{path_on_pc}/{s3_file_path}'
        try:
            try:
                if s3.head_object(Bucket=bucket_name,Key=s3_file_path)['ResponseMetadata']['HTTPStatusCode'] == 200:
                    print(f'{s3_file_path} the file has already been uploaded!')
            except:
                s3.upload_file(pc_file_path, bucket_name, s3_file_path)
                print(f'{s3_file_path} has been uploaded')
        except Exception as e:
            print(e)
            continue
        


In [81]:
upploading_missing_data_to_storage(bucket_name,path_on_pc)

entries/year=2023/month=12/04.csv has been uploaded
entries/year=2023/month=12/06.csv has been uploaded
entries/year=2023/month=12/12.csv has been uploaded
entries/year=2023/month=12/14.csv has been uploaded
entries/year=2023/month=12/21.csv has been uploaded
entries/year=2023/month=12/28.csv has been uploaded
entries/year=2023/month=12/29.csv has been uploaded
entries/year=2024/month=01/19.csv has been uploaded
entries/year=2024/month=01/22.csv has been uploaded
entries/year=2024/month=02/01.csv has been uploaded
entries/year=2024/month=02/05.csv has been uploaded
entries/year=2024/month=02/08.csv has been uploaded
entries/year=2024/month=02/10.csv has been uploaded
entries/year=2024/month=02/13.csv has been uploaded
entries/year=2024/month=02/15.csv has been uploaded
entries/year=2024/month=02/21.csv has been uploaded
entries/year=2024/month=02/24.csv has been uploaded
entries/year=2024/month=03/01.csv has been uploaded
entries/year=2024/month=03/14.csv has been uploaded
entries/year

In [84]:
# Проверяем

list_of_missing_data(list_of_daily_objects_from_s3(bucket_name,Prefix))

,date


Для копирования  данных из одной папки в другую внутри бакета.
Служит а том числе вместо переименования

In [ ]:
for i in range(0,dates_pd.shape[0]):
    key = dates_pd.loc[i,['date_key']].values[0]
    s3_file_path_old = f'entryes_st_mobile/{key}'
    s3_file_path_new = f'entries_st_mobile/{key}'
    s3.copy_object(
        Bucket=bucket_name,
        CopySource=f'{bucket_name}/{s3_file_path_old}',
        Key=s3_file_path_new
    )